# Nova formulação

Uma nova formulação a partir do artigo

*Duxbury, Lavor, Liberti, Salles-Neto, Unassigned distance geometry and molecular conformation problems, Journal of Global Optimization, v.83, pp: 73-82, 2022.*

A ideia é substituir a função objetivo do modelo (3) no artigo acima por:
$$
    \min \sum_{i=1}^{n-1} \sum_{j=i+1}^{n} \left( \sum_{k=1}^{m} a_{ij}^k \big\vert \Vert x_i - x_j \Vert_2 - d_k \big\vert \right),
$$
onde $x_i = (x_{i,1}, x_{i,2}, x_{i,3})^\mathsf{T}$, $i = 1, 2, \ldots, n$.

In [55]:
%pip install gurobipy

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [56]:
import gurobipy as gp
from gurobipy import GRB

## Preprocessamento

In [57]:
import numpy as np

# Lista de distâncias
n = 5
d = np.sort(
    np.array(
        [
            1.5,
            2.5244,
            3.8724,
            4.8611,
            1.5,
            2.5244,
            3.6242,
            1.5,
            2.5244,
            1.5,
        ]
    )
)
m = len(d)


Sejam
$$
    D = \max_{k=1,\ldots, m} \{ d_k \}
$$

In [58]:
D = d.max()


## Modelo

### Variáveis

In [59]:
model = gp.Model("UnassignedDistance")
model.setParam(GRB.Param.NonConvex, 2)

# Variável com as coordenadas
x = {
    i: model.addMVar(3, name=f"x[{i}]", vtype=GRB.CONTINUOUS)
    for i in range(n)
}

# Vetor de distância entre os átomos i e j
v = {
    (i, j): model.addMVar(3, name=f"v[{i},{j}]", vtype=GRB.CONTINUOUS)
    for i in range(n - 1)
    for j in range(i + 1, n)
}

# Erro nas distâncias
alpha = model.addVars(m, name="alpha")

# Valor absoluto do erro das arestas
y = model.addVars(m, name="y")

# Distância entre os átomos i e j
t = {
    (i, j): model.addVar(name=f"w[{i},{j}]", vtype=GRB.CONTINUOUS, lb=0)
    for i in range(n - 1)
    for j in range(i + 1, n)
}

# Variável de decisão (distância k é referente ao par i, j.)
a = {
    (i, j, k): model.addVar(name=f"a[{i},{j},{k}]", vtype=GRB.BINARY)
    for i in range(n - 1)
    for j in range(i + 1, n)
    for k in range(m)
}

# Variável z (distância entre os átomos i e j se distância k é referente ao par i, j. Zero em caso contrário.)
z = {
    (i, j, k): model.addVar(name=f"z[{i},{j},{k}]", vtype=GRB.CONTINUOUS)
    for i in range(n - 1)
    for j in range(i + 1, n)
    for k in range(m)
}


Set parameter NonConvex to value 2


### Função objetivo

O novo modelo:
$$
    \text{(P):} \quad \min \sum_{k=1}^{m} y_k
$$

In [60]:
model.setObjective(gp.quicksum(y), GRB.MINIMIZE)

### Restrições

O problema está sujeito a:
$$
\begin{aligned}
    &\text{(C1):} & y_k &\geq \alpha_k, \quad y_k \geq -\alpha_k \\
    &\text{(C2):} & t_{ij}^2 &= \Vert x_i - x_j \Vert_2^2 \\
    &\text{(C3):} & -(1-a_{ij}^k)D + t_{ij} &\leq z_{ij}^k \leq t_{ij} + (1-a_{ij}^k)D \\
    &\text{(C4):} & -a_{ij}^kD &\leq z_{ij}^k \leq a_{ij}^kD \\
    &\text{(C5):} & -(1-a_{ij}^k)D + (d_k + \alpha_k) &\leq z_{ij}^k \leq (1-a_{ij}^k)D + (d_k + \alpha_k)
\end{aligned}
$$
para $i = 1, 2, \ldots, n−1,\; j = i+1, i+2, \ldots, n,\; k = 1, 2, \ldots, m$

In [61]:
# C1
c1a = model.addConstrs((y[k] >= alpha[k] for k in range(m)), name="C1a")
c1b = model.addConstrs((y[k] >= -alpha[k] for k in range(m)), name="C1b")

# C2
c2_aux = model.addConstrs(
    (v[i, j] == x[i] - x[j] for i in range(n - 1) for j in range(i + 1, n)),
    name="C2aux",
)
c2 = model.addConstrs(
    (t[i, j] == gp.norm(v[i, j], 2) for i in range(n - 1) for j in range(i + 1, n)),
    name="C2",
)

# C3
c3a = model.addConstrs(
    (
        -(1 - a[i, j, k]) * D + t[i, j] <= z[i, j, k]
        for i in range(n - 1)
        for j in range(i + 1, n)
        for k in range(m)
    ),
    name="C3a",
)
c3b = model.addConstrs(
    (
        z[i, j, k] <= (1 - a[i, j, k]) * D + t[i, j]
        for i in range(n - 1)
        for j in range(i + 1, n)
        for k in range(m)
    ),
    name="C3b",
)

# C4
c4a = model.addConstrs(
    (
        -a[i, j, k] * D <= z[i, j, k]
        for i in range(n - 1)
        for j in range(i + 1, n)
        for k in range(m)
    ),
    name="C4a",
)
c4b = model.addConstrs(
    (
        z[i, j, k] <= a[i, j, k] * D
        for i in range(n - 1)
        for j in range(i + 1, n)
        for k in range(m)
    ),
    name="C4b",
)

# C5
c5a = model.addConstrs(
    (
        -(1 - a[i, j, k]) * D + (d[k] + alpha[k]) <= z[i, j, k]
        for i in range(n - 1)
        for j in range(i + 1, n)
        for k in range(m)
    ),
    name="C5a",
)
c5b = model.addConstrs(
    (
        z[i, j, k] <= (1 - a[i, j, k]) * D + (d[k] + alpha[k])
        for i in range(n - 1)
        for j in range(i + 1, n)
        for k in range(m)
    ),
    name="C5b",
)


Mantendo as restrições do modelo (3):
$$
\begin{aligned}
    &\text{(C6):} & \sum_{i=1}^{n-1} \sum_{j=1+1}^{n} a_{ij}^k &= 1 && k = 1, 2, \ldots, m, \\
    &\text{(C7):} & \sum_{k=1}^{m} a_{ij}^k &\leq 1 && i = 1, 2, \ldots, n−1,\; j = i+1, i+2, \ldots, n,
\end{aligned}
$$

In [62]:
# C6
c5 = model.addConstrs(
    gp.quicksum(a[i, j, k] for i in range(n - 1) for j in range(i + 1, n)) == 1
    for k in range(m)
)
# C7
c6 = model.addConstrs(
    gp.quicksum(a[i, j, k] for k in range(m)) <= 1
    for i in range(n - 1)
    for j in range(i + 1, n)
)


## Solução

In [63]:
# Salva a formulação do modelo
model.write("unassigned_distance.lp")

# Otimiza o modelo
model.setParam("TimeLimit", 5 * 60)
model.setParam("LogToConsole", 0)

model.optimize()


Set parameter TimeLimit to value 300


In [64]:
# Salva os resultados no arquivo out.sol
model.write("out.sol")


## Pós-processamento

In [65]:
%pip install py3Dmol

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [66]:
def gurobi_variables_to_xyz_str(x: list) -> str:
    xyz_coords = ["C   " + "   ".join(str(x_i_l.X) for x_i_l in x_i) for x_i in x.values()]
    return "\n".join([str(n), "OUTPUT", *xyz_coords])


In [67]:
xyz_str = gurobi_variables_to_xyz_str(x)

with open("output.xyz", "w") as f:
    f.write(xyz_str)

print(xyz_str)

4
OUTPUT
C   3.609783421319821   1.4017648725495795   0.0
C   2.1098450858577915   1.3860552468956546   0.0
C   1.4999183292673934   0.01565780379394327   0.0
C   0.0   0.0   0.0


In [68]:
import py3Dmol
view = py3Dmol.view(data=xyz_str)
view.setBackgroundColor('000')
view.setStyle({'stick':{}})
view

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol